In [1]:
import cv2

In [2]:
cap = cv2.VideoCapture("D:/main project/videoplayback1.mp4")

In [3]:
ret, frame1 = cap.read()
ret, frame2 = cap.read()

In [ ]:
while cap.isOpened():
    diff = cv2.absdiff(frame1, frame2) 
    gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY) 
    blur = cv2.GaussianBlur(gray, (5, 5), 0) 
    _, thresh = cv2.threshold(blur, 20, 255, cv2.THRESH_BINARY) 
    dilated = cv2.dilate(thresh, None, iterations=3) 
    contours, _ = cv2.findContours(dilated, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 

    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)

        if cv2.contourArea(contour) < 900:
            continue

        cv2.rectangle(frame1, (x, y), (x+w, y+h), (0, 255, 0), 2)

    cv2.imshow('Frame Difference', frame1) 
    frame1 = frame2
    ret, frame2 = cap.read()

    if cv2.waitKey(40) == 27: 
        break

cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:650: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function 'cv::arithm_op'


: 

In [7]:

# Create a background subtractor object
fgbg = cv2.createBackgroundSubtractorMOG2() 

# Open the video file
cap = cv2.VideoCapture("D:/main project/videoplayback.mp4")

while(1):
    # Read a frame from the video
    ret, frame = cap.read()

    # Apply background subtraction
    fgmask = fgbg.apply(frame) 

    # Display the resulting frame
    cv2.imshow('frame',fgmask)

    # Exit on pressing 'q'
    k = cv2.waitKey(30) & 0xff
    if k == ord('q'):
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()

In [25]:
import cv2

def keep_subject_visible(video_path):
  """
  Performs frame subtraction on a video and attempts to keep the subject visible.

  Args:
    video_path: Path to the input video file.

  Returns:
    None
  """

  cap = cv2.VideoCapture(video_path)

  # Get initial frame for background estimation
  ret, prev_frame = cap.read()
  gray_prev = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

  while(True):
    ret, frame = cap.read()
    if not ret:
      break

    gray_curr = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Calculate frame difference
    frame_diff = cv2.absdiff(gray_prev, gray_curr)

    # Threshold the difference image to get a binary mask
    _, thresh = cv2.threshold(frame_diff, 25, 255, cv2.THRESH_BINARY)

    # Apply morphological operations to clean the mask (optional)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # Invert the mask to get the subject
    #mask = cv2.bitwise_not(thresh)

    # Apply the mask to the current frame
    #result = cv2.bitwise_and(frame, frame, mask=mask)
    result = cv2.bitwise_and(frame, frame, mask=thresh)

    # Display the result
    cv2.imshow('Subject', result)

    # Update previous frame
    gray_prev = gray_curr

    if cv2.waitKey(30) & 0xFF == ord('q'):
      break

  cap.release()
  cv2.destroyAllWindows()

# Example usage:
video_path = "D:/main project/videoplayback.mp4"
keep_subject_visible(video_path)

In [26]:
import cv2

def isolate_subject(video_path):
  cap = cv2.VideoCapture(video_path)

  while True:
    ret, frame = cap.read()
    if not ret:
      break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply thresholding (replace with an appropriate thresholding method)
    thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)[1]

    # Inpaint the background (replace with an inpainting library/function)
    inpainted_frame = cv2.inpaint(frame, thresh, 3, cv2.INPAINT_TELEA)

    # Display the result
    cv2.imshow('Subject', inpainted_frame)

    if cv2.waitKey(30) & 0xFF == ord('q'):
      break

  cap.release()
  cv2.destroyAllWindows()

# Example usage:
video_path = "D:/main project/videoplayback.mp4"
isolate_subject(video_path)

In [27]:
import cv2

def keep_subject_visible_mog2(video_path):
  """
  Performs frame subtraction with MOG2 and attempts to keep the subject visible.

  Args:
    video_path: Path to the input video file.

  Returns:
    None
  """

  cap = cv2.VideoCapture(video_path)

  # Create MOG2 background subtractor
  fgbg = cv2.createBackgroundSubtractorMOG2()

  # Read first frame for initialization
  ret, frame = cap.read()
  if not ret:
    print("Error: Could not read first frame.")
    return

  # Initialize background model
  fgmask = fgbg.apply(frame)

  while(True):
    ret, frame = cap.read()
    if not ret:
      break

    # Update background model
    fgmask = fgbg.apply(frame)

    # Apply thresholding to foreground mask (optional)
    _, thresh = cv2.threshold(fgmask, 25, 255, cv2.THRESH_BINARY)

    # Apply morphological operations to refine mask (optional)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # Invert the mask to isolate the subject (optional)
    # mask = cv2.bitwise_not(thresh)

    # Apply the mask to the current frame (consider both options)
    # Option 1: Mask out background (might not preserve subject color well)
    # result = cv2.bitwise_and(frame, frame, mask=thresh)
    # Option 2: Keep subject color, inpaint background (might require additional steps)
    result = frame.copy()
    result[thresh == 0] = (0, 0, 0)  # Set background pixels to black

    # Display the result
    cv2.imshow('Subject', result)

    if cv2.waitKey(30) & 0xFF == ord('q'):
      break

  cap.release()
  cv2.destroyAllWindows()

# Example usage:
video_path = "D:/main project/videoplayback.mp4"
keep_subject_visible_mog2(video_path)

In [1]:
import cv2

def keep_subject_visible_and_crop(video_path, output_video_path, crop_x, crop_y, crop_width, crop_height):
  """
  Crops a video while attempting to keep the subject visible using frame subtraction.

  Args:
    video_path: Path to the input video file.
    output_video_path: Path to save the cropped video.
    crop_x: x-coordinate of the top-left corner of the cropping region.
    crop_y: y-coordinate of the top-left corner of the cropping region.
    crop_width: Width of the cropping region.
    crop_height: Height of the cropping region.
  """

  cap = cv2.VideoCapture(video_path)

  # Check if video opened successfully
  if not cap.isOpened():
    print("Error opening video:", video_path)
    return

  # Get video properties
  frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
  frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fps = cap.get(cv2.CAP_PROP_FPS)

  # Validate crop region within frame dimensions
  if crop_x + crop_width > frame_width or crop_y + crop_height > frame_height:
    print("Error: Crop region exceeds frame boundaries. Adjust coordinates.")
    return

  # Define video writer for output
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  out = cv2.VideoWriter(output_video_path, fourcc, fps, (crop_width, crop_height))

  # Create MOG2 background subtractor
  fgbg = cv2.createBackgroundSubtractorMOG2()

  # Read first frame for initialization
  ret, frame = cap.read()
  if not ret:
    print("Error: Could not read first frame.")
    return

  # Initialize background model
  fgmask = fgbg.apply(frame)

  while(True):
    ret, frame = cap.read()
    if not ret:
      break

    # Update background model
    fgmask = fgbg.apply(frame)

    # Apply thresholding to foreground mask (optional)
    _, thresh = cv2.threshold(fgmask, 25, 255, cv2.THRESH_BINARY)

    # Apply morphological operations to refine mask (optional)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

    # Find contours in the foreground mask (assuming subject is a single object)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Check if a contour is found
    if len(contours) > 0:
      # Get the largest contour (assuming the subject is the largest object)
      largest_contour = max(contours, key=cv2.contourArea)

      # Get bounding rectangle of the largest contour
      x, y, w, h = cv2.boundingRect(largest_contour)

      # Consider intersection of subject bounding box and crop region
      crop_x1 = max(crop_x, x)  # Adjust crop_x if subject is partially outside
      crop_y1 = max(crop_y, y)  # Adjust crop_y if subject is partially outside
      crop_width1 = min(crop_width, x + w - crop_x1)  # Adjust crop width if subject is smaller
      crop_height1 = min(crop_height, y + h - crop_y1)  # Adjust crop height if subject is smaller

      # Ensure crop dimensions are valid
      if crop_width1 <= 0 or crop_height1 <= 0:
        print("Warning: Invalid crop dimensions. Using original crop region.")
        cropped_frame = frame[crop_y:crop_y+crop_height, crop_x:crop_x+crop_width]
      else:
        cropped_frame = frame[crop_y1:crop_y1+crop_height1, crop_x1:crop_x1+crop_width1]

    else:
      # Handle case when no subject is found
      print("Warning: No subject found in frame. Using original crop region.")
      cropped_frame = frame[crop_y:crop_y+crop_height, crop_x:crop_x+crop_width]

    # Check if cropped_frame is valid before resizing
    if cropped_frame is not None and cropped_frame.size > 0:
      resized_frame = cv2.resize(cropped_frame, (crop_width, crop_height))
      out.write(resized_frame)
    else:
      print("Warning: Skipping frame due to empty cropped_frame.")

  cap.release()
  out.release()
  cv2.destroyAllWindows()

# Example usage:
input_video = "D:/main project/videoplayback.mp4"
output_video = "cropped_output.mp4"
crop_x = 100  # Adjust these values as needed
crop_y = 50
crop_width = 640
crop_height = 360

keep_subject_visible_and_crop(input_video, output_video, crop_x, crop_y, crop_width, crop_height)

ModuleNotFoundError: No module named 'vpi'